In [1]:
import numpy as np
import cv2
import os
import math
from matplotlib import pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view

In [2]:
images_names = os.listdir('D:/Milan/Desktop/projects/CapStoneProject/images')
images_names

['i1.jpg', 'i2.jpg', 'i3.jpeg']

In [3]:
images = np.array([cv2.imread(f'D:/Milan/Desktop/projects/CapStoneProject/images/{img}') for img in images_names[2:] ])
# images = [np.array([
#   [[142, 85, 230], [200, 15, 123], [50, 255, 90]], 
#   [[77, 180, 240], [220, 50, 10], [100, 75, 255]],   
#   [[180, 200, 30], [15, 90, 140], [250, 60, 180]]   
# ])]

In [4]:
np.array(images).shape

(1, 168, 300, 3)

In [63]:
def save_bit_planes(bit_planes, output_prefix):
    for i, bit_plane in enumerate(bit_planes[0]):
        print(cv2.imwrite(f"{output_prefix}_bit_plane_{7-i}.jpg", bit_plane))
    

    
def save_amino_acid_to_text_file(matrices,i):
    channel_initial = ['r','b','g']
    for j,channel in enumerate(matrices):
        rows, columns = np.array(channel).shape
        file_path =(f"i{i}_{channel_initial[j]}_amino_acid.txt")
        with open(file_path, 'w') as file:
            for row in channel:
                line = ''.join(map(str, row))
                file.write(line + '\n')
                
                
        
def codon_to_amino_acid():
    codon_dict = {
        'UUU': 'A', 'UUC': 'A',  # A
        'UUA': 'B', 'UUG': 'B',  # B
        'UCU': 'C', 'UCC': 'C', 'UCA': 'C', 'UCG': 'C',  # C
        'UAU': 'D', 'UAC': 'D',  # D
        'UGU': 'E', 'UGC': 'E',  # E
        'UGG': 'F',  # F
        'CUU': 'G', 'CUC': 'G', 'CUA': 'G', 'CUG': 'G',  # G
        'CCU': 'H', 'CCC': 'H', 'CCA': 'H', 'CCG': 'H',    # H
        'CAU': 'I', 'CAC': 'I',  # I
        'CAA': 'J', 'CAG': 'J',  # J
        'CGU': 'K', 'CGC': 'K', 'CGA': 'K', 'CGG': 'K',  # K
        'AUU': 'L', 'AUC': 'L', 'AUA': 'L',  # L
        'ACU': 'M', 'ACC': 'M', 'ACA': 'M', 'ACG': 'M', # M
        'AAU': 'N', 'AAC': 'N',  # N
        'AAA': 'O', 'AAG': 'O',  # O
        'AGU': 'P', 'AGC': 'P',  # P
        'AGA': 'Q', 'AGG': 'Q',  # Q
        'GUU': 'R', 'GUC': 'R', 'GUA': 'R', 'GUG': 'R',  # R
        'GCU': 'S', 'GCC': 'S', 'GCA': 'S', 'GCG': 'S',  # S
        'GAU': 'T', 'GAC': 'T',  # T
        'GAA': 'U', 'GAG': 'U',  # U
        'GGU': 'V', 'GGC': 'V', 'GGA': 'V', 'GGG': 'V',  # V
        'UAA': 'Z', 'UAG': 'Z', 'UGA': 'Z', 'AUG': 'Z',  # Z
    }
    return codon_dict

def get_amino_acid(codon):
    codon_dict = codon_to_amino_acid()
    return codon_dict.get(codon.upper(), "Invalid codon")


def make_amino_acid_matrix(matrices):
    amino_acid_matrices = []
    for channel in matrices:
        rows, columns = channel.shape[1:]
        amino_acid_matrix = [[0 for _ in range(columns)] for _ in range(rows)]
        for x in range(rows):
            for y in range(columns):
                codon = (channel[0][x][y] + channel[1][x][y] + channel[2][x][y])
                amino_acid_matrix[x][y]=get_amino_acid(codon)
        amino_acid_matrices.append(amino_acid_matrix)
    return amino_acid_matrices


def bits_to_dna_matrix(b1, b2, rule):
    char_map = {
        1: 'ACGU', 2: 'AGCU', 3: 'CAUG', 4: 'GAUC', 5: 'CUAG', 
        6: 'GUAC', 7: 'UCGA', 8: 'UGCA'
    }
    return char_map.get(rule, "Enter valid rule number.")[(2 * b1 + b2)]



def make_DNA_matrices(bit_planes, rule):
    matrices= []
    for channel in bit_planes:
        rows, columns = channel.shape[1:]
        matrix = np.array([np.empty((rows,columns), dtype='U6') for _ in range(3)])
        for x in range(rows):
            for y in range(columns):
                for j in range(3):
                    j1 = 2 * j + 1
                    j2 = 2 * j
                    matrix[j][x, y] = bits_to_dna_matrix(channel[j1][x][y], channel[j2][x][y], rule)
        matrices.append(matrix)
    return matrices



def split_image_into_bit_planes(image):
    rows, columns = image.shape
    bit_plane = np.array([np.zeros((rows,columns), dtype=np.uint8) for _ in range(6)])
    for i in range(2,8):
        bit_plane[i-2] = (image>>i) & 1
#         bit_plane[i-2]*=255
    return bit_plane


def l3p(amino_acid_matrices):
    multiplier = np.array([
        [1,2,4],
        [128,0,8],
        [64,32,16]
    ])
    amino_acid_matrices = amino_acid_matrices[0]
    l3pMatrix = []
    for i in range(1,amino_acid_matrices.shape[0]-1):
        array = []
        for j in range(1, amino_acid_matrices.shape[1]-1):
            Pc = amino_acid_matrices[i][j]
            filter_ = np.array([
                [[Pc]*3]*3
            ], dtype='U6')
#             print(amino_acid_matrices.shape)
            comparision = (filter_ > amino_acid_matrices[i-1:i+2,j-1:j+2]).astype('int')
#             print(comparision)
            value = np.sum(comparision@multiplier)
            array.append(value)
        l3pMatrix.append(array)
    return l3pMatrix



def l3p_alternate(amino_acid_matrices):
    # Define the multiplier matrix
    multiplier = np.array([
        [1, 2, 4],
        [128, 0, 8],
        [64, 32, 16]
    ])
    amino_acid_matrices= np.array(amino_acid_matrices[0])

    # Get a 3x3 sliding window view of the input matrix
    windowed = sliding_window_view(amino_acid_matrices, (3, 3))
    print(windowed)

    # Get the center elements (1,1) from each window for comparison
    Pc = amino_acid_matrices[1:amino_acid_matrices.shape[0]-1, 1:amino_acid_matrices.shape[1]-1]

    Center = Pc[..., np.newaxis, np.newaxis]
    print(Center)

    # Perform the comparison using broadcasting: Pc > window
    comparison = (Center < windowed).astype(int)
    print(comparison)

    # Reshape the multiplier to be broadcastable with the comparison array
    multiplier = multiplier.reshape((1, 1, 3, 3))
#     print(multiplier)
#     print()

    # Compute the dot product along the last two dimensions (3x3 window)
    print(comparison*multiplier)
    l3pMatrix = np.sum(comparison * multiplier, axis=(-2, -1))

    return l3pMatrix

            
    

def create_annular_ring_matrix(amino_acid_matrices,k):
    print(np.array(amino_acid_matrices).shape)
    rows = np.array(amino_acid_matrices).shape[1]
    cols = np.array(amino_acid_matrices).shape[2]
    row_center = (rows-1)//2
    col_center = (cols-1)//2
    point1 = np.array((row_center,col_center))
    point2 = np.array((rows-1,cols-1))+0.0000001
#     point3 = np.array((0,cols))
    max_distance = np.linalg.norm(point1-point2)
#     max_distance = max(max_distance,np.linalg.norm(point1-point3))
    annular_distance = max_distance/k
    annular_ring_matrices = []
    for channel in amino_acid_matrices:
        annular_ring_matrix = [[] for _ in range(k)]
        for i in range(rows):
            for j in range(cols):
                point1 = np.array((row_center,col_center))
                point2 = np.array((i,j))
                distance = np.linalg.norm(point1-point2)
                ring_number = (distance // annular_distance).astype(int)
#                 print(ring_number)
                annular_ring_matrix[ring_number].append(channel[i][j])
        annular_ring_matrices.append(annular_ring_matrix)
    return annular_ring_matrices
        
        
        

def Encoding_images(images, rule):
    for i,image in enumerate(images):
        bit_planes = []
        DNA_matrices = []
        amino_acid_matrices = []
        r_channel, g_channel, b_channel = image[:,:,2] , image[:,:,1],image[:,:,0]
        channel_list = [r_channel, g_channel, b_channel]
        for m,channel in enumerate(channel_list):
            print(m)
            bit_planes.append(split_image_into_bit_planes(channel))
        print("bit_planes_bangaya")
        DNA_matrices = make_DNA_matrices(bit_planes, rule)
        print("DNA_Matrix_bangaya")
        amino_acid_matrices = make_amino_acid_matrix(DNA_matrices)
        print("AA_bangaya")
        save_amino_acid_to_text_file(amino_acid_matrices,i)
        print("Save Ho Gaya")
        k=4
        annular_ring_matrix = create_annular_ring_matrix(amino_acid_matrices,k)
#         l3pMatrix = l3p(np.array(amino_acid_matrices))
#         l3pMatrix = l3p_alternate(amino_acid_matrices)
        
#         print(l3pMatrix)
#         save_bit_planes(bit_planes, f"output{i}")
#     print((bit_planes))

In [64]:
Encoding_images(images,7)
# images stored at :  C:\Users\milan\CapStone_Project

0
1
2
bit_planes_bangaya
DNA_Matrix_bangaya
AA_bangaya
Save Ho Gaya
(3, 168, 300)


# Colour channel wala overwrite ho rha hai

```python
     print(channel[j1][x][y],",",channel[j2][x][y])
                    matrix[j][x, y] = bits_to_dna_matrix(channel[j1][x][y], channel[j2][x][y], rule)
    print(matrix)
```


In [7]:
mat1 = np.array([['E' ,'E', 'V'],
   ['D', 'E' ,'H'],
   ['V', 'L', 'E']], dtype = 'U6')

In [8]:
mat2 = np.array([['E']],dtype='U6')

In [9]:
ann = []
k=3
ann1 = [ann.extend([1,]) for i in range(k)]
ann1

[None, None, None]

In [10]:
l = [[] for _ in range(3)]
l

[[], [], []]